In [ ]:
!pip install classiq

In [ ]:
from classiq import (
    Model,
    RegisterUserInput,
    construct_grover_model,
    execute,
    set_constraints,
    show,
    synthesize,
)
from classiq.builtin_functions import ArithmeticOracle, GroverOperator, StatePreparation
from classiq.execution import ExecutionDetails

**Game Board**

|m/n|x1|x2|x3|x4|
|---|---|---|---|---|
|y1|2|-|-|-|
|y2|-|-|3|-|
|y3|-|4|-|-|
|y4|-|-|-|1|

Notes: 
- each space can only contain one number
- (1,1), (2,3), (3,2), (4,4) already taken up
- for each number, it may be present only once per: row, column, 2x2 matrix
  - ie. no same number may be present in a row, column, or 2x2 matrix
- given the board, since the 4 preset numbers are unique and placed in separate matrix: possible placement options for each number is 5 (16 total - 4 placed in each matrix - (4+3) in corresponding row+column), and total placement required for each number is 3
  - ie. 3 out of 5 placements for each number must be valid: need to find the correct combinations 5C3 for 4 numbers

**Coversion to SAT problem**

|x-axis|m1|m2|m3|m4|
|---|---|---|---|---|
|1|x1|x2|x3|-|
|2|-|x4|x5|x6|
|3|x7|x8|-|x9|
|4|x10|-|x11|x12|

CNF: f(x) = (
((x1) ^ (x2) ^ (x3)) and ((x4) ^ (x5) ^ (x6)) and ((x7) ^ (x8) ^ (x9)) and ((x10) ^ (x11) ^ (x12)) and 
((x1) ^ (x7) ^ (x10)) and ((x2) ^ (x4) ^ (x8)) and ((x3) ^ (x5) ^ (x11)) and ((x6) ^ (x9) ^ (x12)) 
)

|y-axis|n1|n2|n3|n4|
|---|---|---|---|---|
|1|y1|y2|y3|-|
|2|-|y4|y5|y6|
|3|y7|-|y8|y9|
|4|y10|y11|-|y12|

CNF: f(y) = (
((y1) ^ (y2) ^ (y3)) and ((y4) ^ (y5) ^ (y6)) and ((y7) ^ (y8) ^ (y9)) and ((y10) ^ (y11) ^ (y12)) and 
((y1) ^ (y7) ^ (y10)) and ((y2) ^ (y4) ^ (y11)) and ((y3) ^ (y5) ^ (y8)) and ((y6) ^ (y9) ^ (y12)) 
)

|(x,y)|(1,1)|(1,2)|(1,3)|(1,4)|(2,1)|(2,2)|(2,3)|(2,4)|(3,1)|(3,2)|(3,3)|(3,4)|(4,1)|(4,2)|(4,3)|(4,4)|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|1|taken|xy1|xy2|-|xy3|xy4|taken|-|xy5|taken|-|-|-|-|-|taken|
|2|taken|-|-|-|-|-|taken|xy6|-|taken|xy7|xy8|-|xy9|xy10|taken|
|3|taken|-|xy11|xy12|xy13|-|taken|xy14|-|taken|-|-|-|-|xy15|taken|
|4|taken|xy16|-|-|-|-|taken|-|xy17|taken|-|x18|x19|x20|-|taken|

CNF: f(xy) = (

  ((xy1) ^ (xy2)) and ((xy3) ^ (xy4)) and ((xy5)) and 
  ((xy6)) and ((xy7) ^ (xy8)) and ((xy9)) ^ ((xy10)) and
  ((xy11) ^ (xy12)) and ((xy13) ^ (xy14)) and ((xy15)) and
  ((xy16)) and ((xy17)) ^ ((xy18)) and ((xy19) ^ (xy20)) and

  ((xy1) ^ (xy16)) and ((xy2) ^ (xy11)) and ((xy12)) and 
  ((xy3) ^ (xy13)) and ((xy4)) and ((xy6)) ^ ((xy14)) and
  ((xy5) ^ (xy17)) and ((xy7)) and ((xy8) ^ (xy18)) and
  ((xy19)) and ((xy9)) ^ ((xy20)) and ((xy10) ^ (xy15)) and

)

In [ ]:
# formula = "( ((x1) ^ (x2) ^ (x3)) and ((x4) ^ (x5) ^ (x6)) and ((x7) ^ (x8) ^ (x9)) and ((x10) ^ (x11) ^ (12)) and ((x4) ^ (x7) ^ (x10)) and ((x1) ^ (x8) ^ (x11)) and ((x2) ^ (x5) ^ (x12)) and ((x3) ^ (x6) ^ (x9)) )"

register_size = RegisterUserInput(size=1)

model = construct_grover_model(
    expression=formula,
    definitions=[
        ("x1", register_size),
        ("x2", register_size),
        ("x3", register_size),
        ("x4", register_size),
        ("x5", register_size),
        ("x6", register_size),
        ("x7", register_size),
        ("x8", register_size),
        ("x9", register_size),
        ("x10", register_size),
        ("x11", register_size),
        ("x12", register_size),
    ],
    uncomputation_method="optimized",  
    num_reps=1,
)

In [ ]:
qprog = synthesize(model)
show(qprog)

In [ ]:
result = execute(qprog).result()
res = result[0].value
registers = ("x1", "x2", "x3","x4", "x5", "x6")
res_by_regs = res.counts_of_multiple_outputs(registers)
most_common_res = max(res_by_regs, key=res_by_regs.get)
dict(zip(registers, most_common_res))